<a href="https://colab.research.google.com/github/abxda/UP_Python_2025/blob/main/Semana_05_02_Lunes_UP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# SECCIÓN 1: INTRODUCCIÓN A LOS EMBEDDINGS DE TEXTO

¿Qué son los Embeddings de Texto?

Imagina que quieres que una computadora "entienda" el significado de las palabras y frases
de la misma forma (o de una forma aproximada) a como lo hacemos los humanos.
Los embeddings de texto son una manera de lograr esto.

En esencia, un embedding de texto es una representación numérica de una porción de texto
(puede ser una palabra, una frase, un párrafo o incluso un documento entero).
Esta representación es un vector de números de punto flotante (por ejemplo, un array de 384, 768 o 1024 números).

Características Clave de los Embeddings:
1.  **Densos**: A diferencia de representaciones más antiguas como "one-hot encoding" que son dispersas
    (muchos ceros), los embeddings son vectores densos (la mayoría de los valores son no nulos).
2.  **Semántica Capturada**: Lo más importante es que estos vectores están diseñados de tal manera
    que textos con significados similares tendrán vectores numéricos (embeddings) cercanos
    en el espacio vectorial. Por ejemplo, el embedding de "rey" estará cerca del embedding de "reina",
    y el embedding de "perro" estará cerca del de "gato".
3.  **Dimensionalidad**: La cantidad de números en el vector se llama la "dimensionalidad" del embedding.
    Modelos más grandes y complejos suelen producir embeddings de mayor dimensionalidad.

¿Por qué son útiles?
Los embeddings permiten a los algoritmos de machine learning trabajar con texto de una manera
mucho más sofisticada:
-   **Búsqueda Semántica**: Encontrar documentos o frases que significan algo similar, no solo
    que comparten las mismas palabras clave.
-   **Clasificación de Texto**: Determinar el tema de un texto, analizar sentimientos (positivo/negativo).
-   **Clustering de Textos**: Agrupar automáticamente textos similares.
-   **Sistemas de Recomendación**: Recomendar artículos o productos basados en descripciones textuales.
-   **Traducción Automática**: Ayudan a mapear significados entre diferentes idiomas.

En este cuaderno, usaremos un modelo moderno llamado BGE-M3 para generar estos embeddings.


In [ ]:
# ==============================================================================
# CELDA 2: SECCIÓN 2 - INSTALACIÓN DE BIBLIOTECAS
# ==============================================================================
# `transformers`: Biblioteca de Hugging Face para modelos de NLP.
# `FlagEmbedding`: Para modelos de embedding específicos como BGE.
# `seaborn`: Para gráficos estéticos.
# `sentence-transformers`: Para una amplia gama de modelos de embedding de frases.
# ------------------------------------------------------------------------------
print("CELDA 2: Instalando bibliotecas...")
!pip install -q transformers
!pip install -q FlagEmbedding
!pip install -q seaborn
!pip install -q sentence-transformers
print("Bibliotecas base instaladas.")


In [ ]:
# ==============================================================================
# CELDA 3: SECCIÓN 2 - IMPORTACIÓN DE BIBLIOTECAS
# ==============================================================================
print("\nCELDA 3: Importando bibliotecas...")
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

# Modelos de embedding
from FlagEmbedding import BGEM3FlagModel
from sentence_transformers import SentenceTransformer

print("Bibliotecas importadas exitosamente.")

In [ ]:
# ==============================================================================
# CELDA 4: SECCIÓN 3 - CARGA DE MODELOS DE EMBEDDINGS
# ==============================================================================
# Se cargarán dos modelos multilingües para comparación.
# Esto puede tardar unos minutos la primera vez debido a la descarga de los modelos.
# ------------------------------------------------------------------------------
print("\nCELDA 4: Cargando modelos de embeddings...")

# --- Modelo 1: BAAI/bge-m3 (FlagEmbedding) ---
print("Cargando el modelo BGE-M3 ('BAAI/bge-m3')...")
try:
    bge_model_instance = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False) # `use_fp16=True` si tienes GPU compatible
    print("Modelo BGE-M3 cargado.")
except Exception as e:
    print(f"Error al cargar BGE-M3: {e}. Asegúrate de tener conexión a internet.")
    bge_model_instance = None

# --- Modelo 2: paraphrase-multilingual-MiniLM-L12-v2 (Sentence-Transformers) ---
st_model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
print(f"\nCargando el modelo Sentence-Transformer: '{st_model_name}'...")
try:
    st_model_instance = SentenceTransformer(st_model_name)
    print(f"Modelo '{st_model_name}' cargado.")
except Exception as e:
    print(f"Error al cargar '{st_model_name}': {e}. Asegúrate de tener conexión a internet.")
    st_model_instance = None

print("\nCarga de modelos completada (o intentada).")


In [ ]:
# ==============================================================================
# CELDA 5: SECCIÓN 4 - DEFINICIÓN DE FUNCIONES PARA GENERAR EMBEDDINGS
# ==============================================================================
print("\nCELDA 5: Definiendo funciones para generar embeddings...")

def generar_embedding_bge(texto, modelo_bge_local):
    """Genera embedding usando BGE-M3."""
    if modelo_bge_local is None: return None
    try:
        return modelo_bge_local.encode([texto])['dense_vecs'][0]
    except Exception as e:
        print(f"Error BGE en '{texto[:20]}...': {e}")
        return None

def generar_embedding_st(texto, modelo_st_local):
    """Genera embedding usando Sentence-Transformer."""
    if modelo_st_local is None: return None
    try:
        return modelo_st_local.encode([texto])[0]
    except Exception as e:
        print(f"Error ST en '{texto[:20]}...': {e}")
        return None
print("Funciones de embedding definidas.")


In [ ]:
# ==============================================================================
# CELDA 6: SECCIÓN 4 - TEXTOS DE EJEMPLO
# ==============================================================================
# Grupos de textos con temas distintos para observar diferencias y similitudes.
# ------------------------------------------------------------------------------
print("\nCELDA 6: Definiendo textos de ejemplo...")
textos_ejemplo = {
    "Deportes": [
        "El equipo local ganó el campeonato de fútbol.",
        "La estrella del baloncesto anotó 50 puntos anoche.",
        "El partido de tenis fue muy emocionante y duró cinco sets."
    ],
    "Tecnología": [
        "La inteligencia artificial está transformando muchas industrias.",
        "El nuevo teléfono inteligente tiene una cámara increíble.",
        "La computación cuántica promete resolver problemas complejos."
    ],
    "Naturaleza y Clima": [
        "El sol brilla intensamente en un cielo despejado.",
        "Se espera una fuerte tormenta de nieve para mañana.",
        "Los bosques tropicales albergan una gran biodiversidad."
    ]
}

lista_todos_textos_global = []
lista_categorias_global = []
for categoria, textos_cat in textos_ejemplo.items():
    for texto in textos_cat:
        lista_todos_textos_global.append(texto)
        lista_categorias_global.append(categoria)

print(f"Total de textos de ejemplo: {len(lista_todos_textos_global)}")
print("Textos de ejemplo definidos.")


In [ ]:
# ==============================================================================
# CELDA 7: SECCIÓN 4 - SELECCIÓN DEL MODELO Y GENERACIÓN DE EMBEDDINGS
# ==============================================================================
# Cambia el valor de MODELO_A_USAR_GLOBAL para probar diferentes modelos.
# Opciones: "BGE-M3" o "SentenceTransformer_MiniLM"
# ------------------------------------------------------------------------------
print("\nCELDA 7: Seleccionando modelo y generando embeddings...")

MODELO_A_USAR_GLOBAL = "SentenceTransformer_MiniLM" #BGE-M3" # O

print(f"--- Usando el modelo: {MODELO_A_USAR_GLOBAL} ---")

lista_embeddings_generados_global = []
if MODELO_A_USAR_GLOBAL == "BGE-M3":
    if bge_model_instance:
        print("Generando embeddings con BGE-M3...")
        for texto in lista_todos_textos_global:
            lista_embeddings_generados_global.append(generar_embedding_bge(texto, bge_model_instance))
    else:
        print("Modelo BGE-M3 no disponible.")
elif MODELO_A_USAR_GLOBAL == "SentenceTransformer_MiniLM":
    if st_model_instance:
        print(f"Generando embeddings con SentenceTransformer ({st_model_name})...")
        for texto in lista_todos_textos_global:
            lista_embeddings_generados_global.append(generar_embedding_st(texto, st_model_instance))
    else:
        print(f"Modelo Sentence-Transformer ({st_model_name}) no disponible.")
else:
    print(f"Modelo '{MODELO_A_USAR_GLOBAL}' no reconocido.")

# Filtrar resultados None (si algún embedding falló) y preparar datos para análisis
textos_validos_global = [lista_todos_textos_global[i] for i, emb in enumerate(lista_embeddings_generados_global) if emb is not None]
categorias_validas_global = [lista_categorias_global[i] for i, emb in enumerate(lista_embeddings_generados_global) if emb is not None]
embeddings_validos_global = [emb for emb in lista_embeddings_generados_global if emb is not None]

matriz_embeddings_global = np.array([]) # Inicializar como array vacío

if embeddings_validos_global:
    matriz_embeddings_global = np.array(embeddings_validos_global)
    print(f"\nMatriz de embeddings creada. Forma: {matriz_embeddings_global.shape}")
    if matriz_embeddings_global.size > 0 : #Verificar si no esta vacio
      print(f"Dimensionalidad del embedding: {matriz_embeddings_global.shape[1]}")
else:
    print("No se pudieron generar embeddings válidos con el modelo seleccionado.")

print("Generación de embeddings (o intento) completada.")


In [ ]:
# ==============================================================================
# CELDA 8: SECCIÓN 5 - VISUALIZACIÓN DE EMBEDDINGS CON T-SNE
# ==============================================================================
# t-SNE reduce la dimensionalidad para visualizar los embeddings en 2D.
# Observa si textos con temas similares se agrupan.
# ------------------------------------------------------------------------------
print("\nCELDA 8: Visualizando embeddings con t-SNE...")

if matriz_embeddings_global.shape[0] > 1: # Se necesita más de un punto
    print("Reduciendo dimensionalidad con t-SNE...")
    tsne_reducer = TSNE(
        n_components=2,
        random_state=42, # Para reproducibilidad
        perplexity=min(5, matriz_embeddings_global.shape[0] - 1), # Ajustar perplexity
        n_iter=500, # Menos iteraciones para ejemplos rápidos
        init='pca',
        learning_rate='auto'
    )
    embeddings_2d_visual = tsne_reducer.fit_transform(matriz_embeddings_global)

    df_tsne_visual = pd.DataFrame({
        'x': embeddings_2d_visual[:, 0], 'y': embeddings_2d_visual[:, 1],
        'texto': textos_validos_global, 'categoria': categorias_validas_global
    })

    plt.figure(figsize=(12, 8)) # Tamaño ajustado
    sns.scatterplot(data=df_tsne_visual, x='x', y='y', hue='categoria', palette='viridis', s=150, alpha=0.9)
    for i in range(df_tsne_visual.shape[0]):
        plt.text(df_tsne_visual.loc[i, 'x'] + 0.03, df_tsne_visual.loc[i, 'y'] + 0.03,
                 df_tsne_visual.loc[i, 'texto'][:25] + "...", # Mostrar menos caracteres
                 fontsize=8, alpha=0.75)

    plt.title(f'Visualización t-SNE (Modelo: {MODELO_A_USAR_GLOBAL})', fontsize=15)
    plt.xlabel('Componente t-SNE 1', fontsize=11)
    plt.ylabel('Componente t-SNE 2', fontsize=11)
    plt.legend(title='Categoría', bbox_to_anchor=(1.02, 1), loc='upper left')
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.tight_layout()
    plt.show()
else:
    print("No hay suficientes embeddings para la visualización t-SNE.")
print("Visualización t-SNE (o intento) completada.")


In [ ]:
# ==============================================================================
# CELDA 9: SECCIÓN 6 - APLICACIÓN: SIMILITUD DE COSENO
# ==============================================================================
# Calculamos cuán similares son los textos entre sí usando sus embeddings.
# La similitud del coseno mide el ángulo entre los vectores de embedding.
# ------------------------------------------------------------------------------
print("\nCELDA 9: Calculando similitud de coseno...")

if matriz_embeddings_global.shape[0] > 1:
    idx_referencia_sim = 0 # Tomamos el primer texto como referencia
    if not textos_validos_global: # Chequeo por si la lista está vacía
        print("No hay textos válidos para calcular similitud.")
    else:
        texto_referencia_sim = textos_validos_global[idx_referencia_sim]
        embedding_referencia_sim = matriz_embeddings_global[idx_referencia_sim].reshape(1, -1)

        print(f"Texto de referencia para similitud: \"{texto_referencia_sim}\" (Modelo: {MODELO_A_USAR_GLOBAL})")

        similitudes_cos = cosine_similarity(embedding_referencia_sim, matriz_embeddings_global)
        similitudes_array_cos = similitudes_cos.flatten()

        df_similitud_cos = pd.DataFrame({
            'texto': textos_validos_global,
            'similitud_con_referencia': similitudes_array_cos,
            'categoria': categorias_validas_global
        })
        df_similitud_ordenada_cos = df_similitud_cos.sort_values(by='similitud_con_referencia', ascending=False)

        print("\nTextos más similares al de referencia (por similitud del coseno):")
        from IPython.display import display
        display(df_similitud_ordenada_cos.head(5)) # Mostrar los 5 más similares
else:
    print("No hay suficientes embeddings para calcular similitudes.")
print("Cálculo de similitud (o intento) completado.")
